# Tensorflow 2 API 개요

## Tensorflow2의 구성과 특징

### Tensorflow v1

만약 텐서플로우같은 딥러닝 프레임워크가 없었다면, 딥러닝에 몸담은 엔지니어들에의 업무 강도가 얼마나 더해지게 될까요? 아마도 여러분은 딥러닝 프레임워크 없이 간단한 딥러닝 모델을 구현해 본 경험이 있으실 것입니다. (혹시 없다면 꼭 한번은 해보시기를 권합니다.) 정말 단순한 Fully Connected Network 만을 구현하는 데도 꽤나 힘든 과정이었을 것입니다. 그 중 가장 어려운 단계는 아무래도 딥러닝 모델의 그래디언트를 수식으로 구하고 그것을 바탕으로 backward propagation을 구현하는 것일 것입니다. 만약 모델이 훨씬 복잡해진다면 그 복잡한 수식의 gradient를 구하기 위해 엄청나게 복잡한 미분식을 다루어야 할텐데 생각만 해도 아찔한 일일 것입니다.

그러나, 일반적으로 우리는 딥러닝 모델을 구현하기 위해 그 모델의 gradient 수식을 계산하느라 골머리를 싸매지 않습니다. PyTorch 등 다른 딥러닝 프레임워크들도 마찬가지이긴 하지만, Tensorflow는 forward propagation 방향의 모델만 설계하면 그 모델의 gradient를 사전에 미리 구해둘 수 있습니다. 이것을 가능하게 하기 위해 Tensorflow는 초기 V1때부터 독특한 설계 사상을 보유했는데, 그것은 바로 Tensorflow를 거대한 노드-엣지 사이의 **유향 비순환 그래프(Directed Acyclic Graph, DAG)**로 정의했다는 점입니다. 그렇게 되면 노드와 노드를 연결하는 매 엣지마다 chain-rule을 기반으로 gradient가 역방향으로 전파될 수 있다는 간단하면서도 강력한 아이디어입니다. 이런 방식을 Tensorflow의 `Graph Mode`라고 합니다.

그러나, 이런 설계가 주는 큰 단점이 하나 있었습니다. Tensorflow V1은 딥러닝 모델을 구성하는 그래프를 그려나가는 부분과, 그 그래프 상에서 연산이 실제 진행되어과는 과정을 엄격하게 분리해 놓았습니다. 거기에 가장 중요한 것이 바로 `session`이라는 개념이었습니다. 그래서 그래프 사이에 벌어지는 모든 연산은 반드시 `session.run()` 안에서 수행되는 것으로 엄격히 제한되었습니다. 물론 이 방식이 주는 대규모 분산환경에서의 확장성과 생산성이라는 장점도 있었지만, Tensorflow V1은 기본적으로 사용하기가 어려웠습니다. 코드도 길고 지저분할 뿐더러, 파이써닉하지 않기 때문에 구현 방식 자체가 난이도가 높았습니다. 무엇보다도, 그래프를 다 만들어놓고 돌려봐야 비로소 모델 구성시의 문제가 드러나는데, 이렇게 문제가 발생했을 때 해결하기가 너무나 어렵고 복잡했기 때문입니다.

### Tensorflow v2

그러나 `PyTorch`는 달랐습니다. PyTorch는 **Eager Mode**라는 것을 제안했는데, 이것은 딥러닝 그래프가 다 그려지지 않아도 얼마든지 부분 실행 및 오류검증이 가능했습니다. 그리고 코드도 간결하고 파이써닉한 설계를 가지고 있었기 때문에 개발자들에게 훨씬 쉽게 다가갈 수 있었습니다. 그래서 마침내 Tensorflow도 Eager Mode를 전격 수용하게 됩니다. 그리고 `Keras`라는 또하나의 매우 쉽고 간결한 머신러닝 프레임워크를 수용하여 Tensorflow v2의 표준 API 로 삼았습니다. 이런 변화가 Tensorflow의 사용방식을 어떻게 바꾸었는지 간단한 예시코드를 보겠습니다. 예시코드는 텐서플로우 튜토리얼에서 가져왔습니다.

```python
# 텐서플로 1.x
outputs = session.run(f(placeholder), feed_dict={placeholder: input})
# 텐서플로 2.0
outputs = f(input)
```

어떻습니까? 기본적으로 V2의 코드가 훨씬 직관적이고 간결합니다. 하지만 여기엔 보이기에의 단순함 이상의 차이가 있습니다. `Session.run()`에 의존하지 않고, 그래프를 완성하지 않아도 부분적인 실행이 가능하다는 Eager Mode의 장점은 <u>설계, 구현, 디버깅 전과정을 매우 쉽고 직관적이게 만들어 줍니다.</u>

그 외에도 Tensorflow V2가 이전 버전에 비해 가지는 다양한 장점들이 있습니다. 아래 글도 한번 읽어보세요.

- [Overview of changes TensorFlow 1.0 vs TensorFlow 2.0
](https://www.datasciencecentral.com/profiles/blogs/tensorflow-1-x-vs-2-x-summary-of-changes)

## Tensorflow2 API로 모델 구성하기

### Tensorflow2 API 알아보기

그럼 가장 간단한 이미지 분류 문제를 풀어가면서 Tensorflow2를 활용한 모델링을 진행해봅시다.

사실 TensorFlow2를 활용함에 있어 딥러닝 모델을 다양한 방법으로 작성할 수 있는데요. 이 부분이 처음 TensorFlow 및 딥러닝을 접하시는 분들에게는 큰 허들이 될 수 있습니다. '하나로 통일해서 하나만 공부하게 해주면 되지.. 왜 이렇게 여러 개를 만들어서 고생을 시키지?' 라는 생각을 하실 수도 있는데요. 처음에야 헷갈리실 수 있지만, 익숙해지시고 나면 경우에 따라 적합한 모델링 방식을 택해서 사용할 수 있다는 점에서 매우 강력하다고 느끼실 수 있을 겁니다.

여기서 소개할 Tensorflow2에서 딥러닝 모델을 작성하는 방법에는 크게 3가지가 있습니다.

- Sequential
- Functional
- Model Subclassing

아마 `Sequential` 모델은 이미 몇번 사용해 보셔서 정확한 개념은 몰라도 익숙하실 것입니다. `Functional`은 `Sequential`의 보다 일반화된 개념입니다. 그리고 `Subclassing`은 클래스로 구현된 기존의 모델을 상속받아 자신만의 모델을 만들어나가는 방식입니다. 세가지 방법 모두 충분히 숙지하여 자유롭게 활용할 수 있게 되는 것이 중요합니다.

### 1) TensorFlow2 Sequential Model

```python
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential()
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)

model.fit(x, y, epochs=10, batch_size=32)
```

지금까지 대부분 공부했던 모델은 위와 같은 형식이었습니다. `model = keras.Sequential()`이라고 먼저 선언하고 layer를 쌓아가는 형식입니다.

- 장점 : 접근하기 쉽다.
- 단점 : 모델의 입력과 출력이 여러개인 경우에는 적합하지 않다.

아래 참고자료에서 Sequential Model로 작성한 코드를 확인해봅시다.

- [텐서플로 2.0 시작하기: 초보자용
](https://www.tensorflow.org/tutorials/quickstart/beginner)

### 2) TensorFlow2 Functional API

```python
import tensorflow as tf
from tensorflow import keras

inputs = keras.Input(shape=(__원하는 입력값 모양__))
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(input)
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)
outputs = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.fit(x,y, epochs=10, batch_size=32)
```

여기서 `Sequential Model`을 활용하는 것과 다른 점은 바로 `keras.Model`을 사용한다는 점입니다. 그래서 `Sequential Model`을 쓰는 것보다 더 일반적인 접근입니다. `Sequential Model`도 `keras.Model`의 한 종류니까요!

`Functional API`는 `Sequential Model`을 활용하는 것 보다 더 자유로운 모델링을 진행할 수 있습니다. *Functional*이라는 뜻과 같이, **입력과 출력을 규정함으로써 모델 전체를 규정**합니다. 그래서 `input`, `output`을 규정할 수 있어 Input, output이 될 수 있는 텐서가 여러개 될수도 있습니다. 

아래 링크에서 자세한 내용을 확인해보세요.

- [Functional API](https://www.tensorflow.org/guide/keras/functional)

### 3) TensorFlow2 Subclassing

```python
import tensorflow as tf
from tensorflow import keras

class CustomModel(keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()

    def call(self, x):
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)

        return x

model = CustomModel()
model.fit(x,y, epochs=10, batch_size=32)
```

마지막으로 `Subclassing`을 활용하면 제일 자유로운 모델링을 할 수 있습니다. 사실 본질적으로는 `Functional`한 접근과 차이가 없습니다. 이것은 `keras.Model`을 상속받은 모델 클래스를 만드는 것이기 때문입니다.

처음 만났던 `Sequential Model`도 따지고보면 `keras.Model`을 상속받은 모델 클래스의 하나일 뿐입니다. `keras.Model`은 위와 같이 `__init__()`이라는 메소드 안에서 레이어 구성을 정의합니다. 그리고 `call()`이라는 메소드 안에서 레이어간 `forward propagationi`을 구현합니다. 

다만 각 레이어에 대한 깉은 이해가 필요하고 초심자에게 의도치않은 버그를 유발할 수 있습니다. 

아래 링크에서 자세한 내용을 확인해보세요.

- [텐서플로 2.0 시작하기 : 전문가용](https://www.tensorflow.org/tutorials/quickstart/advanced)

## Tensorflow2 API로 모델 작성하기 : MNIST (1) - Sequential API 활용

Sequential API를 활용해 MNIST 손글씨 분류기를 만들어봅시다. 아마 굉장히 친숙하실 것 같습니다.

In [1]:
# 필요한 모듈 import
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# 데이터 구성
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [4]:
# Model 구성
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

`Sequential Model`을 만들어보았습니다.

1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)


In [5]:
# 모델 학습 설정
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1087 - accuracy: 0.9668
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0358 - accuracy: 0.9887
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0209 - accuracy: 0.9933
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0146 - accuracy: 0.9952
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0105 - accuracy: 0.9965
313/313 - 2s - loss: 0.0630 - accuracy: 0.9835


[0.06303178519010544, 0.9835000038146973]

## Tensorflow2 API로 모델 작성하기 : MNIST (2) - Functional API 활용

이번에는 `Functional API`를 사용해봅시다. input과 output을 직접 구현해야 합니다.

In [7]:
inputs = keras.Input(shape=(28, 28, 1))

x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = keras.layers.Conv2D(64, 3, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

`inputs`와 `predictions`를 따로 설정해주었습니다. 하나하나 쌓아가는건 비슷해보이죠?

In [8]:
# 모델 학습 설정
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1116 - accuracy: 0.9664
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0346 - accuracy: 0.9893
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0212 - accuracy: 0.9930
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0129 - accuracy: 0.9958
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0092 - accuracy: 0.9971
313/313 - 0s - loss: 0.0422 - accuracy: 0.9879


[0.04221689701080322, 0.9879000186920166]

큰 틀에서 보면 같은 내용이라는게 이해가 가시나요?

## Tensorflow2 API로 모델 작성하기 : MNIST (3) - Subclassing 활용

마지막으로 `subclassing` 방법입니다. `keras.Model`을 상속받은 클래스를 만드는 것입니다. `__init__()` 메소드 안에서 레이어를 선언하고, `call()` 메소드 안에서 `forward propagatioin`을 구현하는 방식이에요.

`Functional` 방식으로 생각해볼 때, `call()`의 입력이 input이고, `call()`의 리턴값이 output입니다.

In [12]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = keras.layers.Conv2D(64, 3, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(128, activation='relu')
        self.fc2 = keras.layers.Dense(10, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

model = CustomModel()

In [13]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1062 - accuracy: 0.9671
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0341 - accuracy: 0.9892
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0197 - accuracy: 0.9935
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0133 - accuracy: 0.9959
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0101 - accuracy: 0.9966
313/313 - 0s - loss: 0.0482 - accuracy: 0.9880


[0.04824186861515045, 0.9879999756813049]

## Tensorflow2 API로 모델 작성 및 학습하기 : CIFAR-100 - (1) Sequential API 활용

이번에는 CIFAR 데이터를 활용해서 모델들을 확인해봅시다!

In [14]:
# 데이터 구성성분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train/255.0, x_test/255.0
print(len(x_train), len(x_test))

50000 10000


In [15]:
# 모델 구성
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(100, activation='softmax')
])

In [16]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 8s 5ms/step - loss: 3.6470 - accuracy: 0.1523
Epoch 2/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.9453 - accuracy: 0.2786
Epoch 3/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.6403 - accuracy: 0.3377
Epoch 4/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.4259 - accuracy: 0.3837
Epoch 5/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.2535 - accuracy: 0.4185
313/313 - 0s - loss: 2.5848 - accuracy: 0.3552


[2.5847561359405518, 0.35519999265670776]

위의 MNIST랑 똑같죠? 마찬가지로 Functional이랑 Subclassing에서도 똑같이 적용해봅시다!

## Tensorflow2 API로 모델 작성 및 학습하기 : CIFAR-100(2) - Functional API 활용


In [17]:
# 모델 작성하기
inputs = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)
x = keras.layers.MaxPool2D((2, 2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2, 2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [18]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 2s 1ms/step - loss: 3.6138 - accuracy: 0.1592
Epoch 2/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.9130 - accuracy: 0.2820
Epoch 3/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.6030 - accuracy: 0.3458
Epoch 4/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.4017 - accuracy: 0.3857
Epoch 5/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.2310 - accuracy: 0.4215
313/313 - 0s - loss: 2.6122 - accuracy: 0.3601


[2.612154483795166, 0.36010000109672546]

## Tensorflow2 API로 모델 작성 및 학습하기 : CIFAR-100 (3) - Subclassing




In [19]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

In [20]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 2s 1ms/step - loss: 3.6090 - accuracy: 0.1571
Epoch 2/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.8981 - accuracy: 0.2846
Epoch 3/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.6039 - accuracy: 0.3409
Epoch 4/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.3844 - accuracy: 0.3888
Epoch 5/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.2165 - accuracy: 0.4257
313/313 - 0s - loss: 2.5828 - accuracy: 0.3637


[2.5827531814575195, 0.3637000024318695]

## Gradient Type 활용

### Autometic differentiation - Gradient Type

우리는 조금 전까지 아주 비슷한 테스크 2개를, 본질적으로 큰 차이가 없는 3개의 모델 구성 방법을 활용하여 딥러닝으로 구현해 보았습니다. 그동안 완전히 동일하게 구성했던 것은 바로 아래와 같이 구성된 모델 학습 관련 부분입니다.

```python
# 모델 학습 설정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
```

Numpy만 가지고 딥러닝을 구현하는 것을 회상해 봅시다. `model.fit()`이라는 한줄로 수행 가능한 딥러닝 모델 훈련 과정은 실제로는 어떠했나요?

- Forward Propagation 수행 및 중간 레이어값 저장
- Loss 값 계산
- 중간 레이어값 및 Loss를 활용한 체인룰(chain rule) 방식의 역전파(Backward Propagation) 수행
- 학습 파라미터 업데이트

이상 4단계로 이루어진 train_step 을 여러번 반복했습니다.

이런 과정이 TF2 API에는 `model.fit()`이라는 메소드 안에 모두 추상화되어 감추어져 있습니다.

Tensorflow에서 제공하는 tf.GradientTape는 위와 같이 순전파(forward pass) 로 진행된 모든 연산의 중간 레이어값을 tap에 기록하고, 이를 이용해 gradient를 계산한 후 **tape**를 폐기하는 기능을 수행합니다. 그러면 아래에서는 이전 스텝에서 진행했던 학습을 `tf.GradientTap`e를 이용한 것으로 변형해 보겠습니다. `tf.GradientTape`는 이후 그래디언트를 좀더 고급스럽게 활용하는 다양한 기법을 통해 자주 만나게 될 것입니다.

In [21]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

50000 10000


In [22]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [23]:
import time

def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            if step % batch_size == batch_size -1:
                x_batch.append(x)
                y_batch.append(y)
                loss = train_step(np.array(x_batch), np.array(y_batch))
                x_batch = []
                y_batch = []
        print("Epoch %d last batch loss = %.4f" % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))
    
train_model()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0 last batch loss = 4.3846
Epoch 1 last batch loss = 4.4301
Epoch 2 last batch loss = 4.5350
Epoch 3 last batch loss = 4.2129
Epoch 4 last batch loss = 3.9236
It took 58.99192023277283 seconds


In [25]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy

1/1 [==============================] - 0s 618us/step


0.0527

잘 동작하는 것을 확인할 수 있습니다. :)

In [26]:
from IPython.core.display import display, HTML
display(HTML("<style> .container{width:90% !important;}</style>"))